In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_F-14-21998CD-40-009.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_A-14-22549AB-40-024.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_TA-14-13200-100-003.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_F-14-9133-400-011.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_F-14-25197-100-026.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_PT-14-22704-100-024.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_A-14-22549AB-40-021.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_F-14-21998CD-100-019.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_A-14-22549AB-40-017.png
/kaggle/input/breast-cancer-dataset/BreaKHis_Total_dataset/benign/SOB_B_F-14-25197-200-051.png
/kaggle/input/breast-cancer-dataset/BreaKHi

In [2]:
!pip install --upgrade tensorflow_hub

In [3]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

Number of replicas: 1
2.4.1


In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH_1 = KaggleDatasets().get_gcs_path('breast-cancer-dataset')
BATCH_SIZE = 30 * strategy.num_replicas_in_sync
IMAGE_SIZE = [180, 180]
EPOCHS = 35
print(BATCH_SIZE)

30


In [9]:
file= tf.io.gfile.glob(str(GCS_PATH_1 + '/BreaKHis_Total_dataset/*/*'))
print(len(file))

7783


In [21]:
train_base,test_base=train_test_split(file,test_size=0.12)
train_data,val_data=train_test_split(train_base,test_size=0.2)


In [11]:
def count_benign_malignant(file):
    print("\n\tTotal images count in training set : " + str(len(file)))
    COUNT_BENIGN = len([filename for filename in file if ("benign" in filename)])
    print("\n\tBenign images count in training set : " + str(COUNT_BENIGN))

    COUNT_MALIGNANT = len([filename for filename in file if ("malignant" in filename)])
    print("\tMalignant images count in training set : " + str(COUNT_MALIGNANT))
    
    if (len(file)==(COUNT_BENIGN+COUNT_MALIGNANT)):
        print("\n\tThe function classified the data correctly ",COUNT_BENIGN+COUNT_MALIGNANT)
    else:
        print("\n\tThe function classification is wrong ",COUNT_BENIGN+COUNT_MALIGNANT)
    return COUNT_BENIGN,COUNT_MALIGNANT

In [22]:
count_benign_malignant(file)
count_benign_malignant(train_base)
count_benign_malignant(test_base)
print("\n\n\n")
count_benign_malignant(train_data)
count_benign_malignant(val_data)


	Total images count in training set : 7783

	Benign images count in training set : 2479
	Malignant images count in training set : 5304

	The function classified the data correctly  7783

	Total images count in training set : 6849

	Benign images count in training set : 2188
	Malignant images count in training set : 4661

	The function classified the data correctly  6849

	Total images count in training set : 934

	Benign images count in training set : 291
	Malignant images count in training set : 643

	The function classified the data correctly  934





	Total images count in training set : 5479

	Benign images count in training set : 1752
	Malignant images count in training set : 3727

	The function classified the data correctly  5479

	Total images count in training set : 1370

	Benign images count in training set : 436
	Malignant images count in training set : 934

	The function classified the data correctly  1370


(436, 934)

In [23]:
def file_info(file):
    list_ds = tf.data.Dataset.from_tensor_slices(file)
    IMG_COUNT = tf.data.experimental.cardinality(list_ds).numpy()
    print("Images count: " + str(IMG_COUNT),"\n")
    
    return list_ds,IMG_COUNT

In [24]:
train_list_ds,TRAIN_IMG_COUNT= file_info(train_data)
val_list_ds,VAL_IMG_COUNT=file_info(val_data)
test_list_ds,TEST_IMG_COUNT=file_info(test_base)

for f in train_list_ds.take(5):
    print(f.numpy())

Images count: 5479 

Images count: 1370 

Images count: 934 

b'gs://kds-5843436483fbe119fe4fd0f287baa6387929394b904d8f12206255b6/BreaKHis_Total_dataset/malignant/SOB_M_DC-14-5694-40-013.png'
b'gs://kds-5843436483fbe119fe4fd0f287baa6387929394b904d8f12206255b6/BreaKHis_Total_dataset/malignant/SOB_M_DC-14-10926-40-006.png'
b'gs://kds-5843436483fbe119fe4fd0f287baa6387929394b904d8f12206255b6/BreaKHis_Total_dataset/benign/SOB_B_PT-14-29315EF-100-009.png'
b'gs://kds-5843436483fbe119fe4fd0f287baa6387929394b904d8f12206255b6/BreaKHis_Total_dataset/malignant/SOB_M_MC-14-16456-40-053.png'
b'gs://kds-5843436483fbe119fe4fd0f287baa6387929394b904d8f12206255b6/BreaKHis_Total_dataset/malignant/SOB_M_MC-14-13413-100-026.png'


In [25]:
def get_label(file_path):

    parts = tf.strings.split(file_path, os.path.sep)
    if (parts[-2]== "malignant"):
        return True
    elif (parts[-2]== "benign"):
        return False
    else:
        return False

In [26]:
def decode_img(img):
    img=tf.image.decode_image(img, channels=3,expand_animations=False)
    img=tf.image.convert_image_dtype(img,tf.float32)
    return tf.image.resize(img, IMAGE_SIZE)

In [27]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [28]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [30]:
def prepare_for_Data_Aug(ds, cache=False,repeat=False,shuffle=False, shuffle_buffer_size=1000):

    if cache: # only for training and validation data set
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
            
    if shuffle: # only for training and validation data set
        ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    
    normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
    ds = ds.map(lambda x, y: (normalization_layer(x), y))
    
    if repeat: # only for training and validation data set
        ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)


    return ds

In [31]:
train_ds = prepare_for_Data_Aug(train_ds,cache=True,repeat=True,shuffle=True,shuffle_buffer_size=(TRAIN_IMG_COUNT//4))

val_ds = prepare_for_Data_Aug(val_ds,cache=True,repeat=True,shuffle=True,shuffle_buffer_size=(VAL_IMG_COUNT//4))

test_ds= prepare_for_Data_Aug(test_ds)

In [32]:
def conv_block(filters):
    
    block = tf.keras.Sequential([
        
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
        
    ])
    
    return block

In [33]:
def dense_block(units, dropout_rate):
    
    block = tf.keras.Sequential([
        
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
        
    ])
    
    return block

In [35]:
def data_aug():
    
    block = tf.keras.Sequential([

            tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal",),
            tf.keras.layers.experimental.preprocessing.RandomFlip("vertical",),
            tf.keras.layers.experimental.preprocessing.RandomRotation(0.1,fill_mode='reflect',),
            tf.keras.layers.experimental.preprocessing.RandomZoom(0.1,),
    ])
    
    return block

In [36]:
def build_model():
    model = tf.keras.Sequential([
        
        tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
        
        data_aug(),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(32),
        conv_block(64),
        
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        
        tf.keras.layers.Dense(1, activation='sigmoid')
        
    ])
    
    return model

In [37]:
with strategy.scope():
    
    model = build_model()

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=METRICS
    )

In [38]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("pneu_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [39]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [40]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 180, 180, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 180, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 16)        0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 45, 45, 32)        2160      
_________________________________________________________________
sequential_2 (Sequential)    (None, 22, 22, 64)        7392      
_________________________________________________________________
sequential_3 (Sequential)    (None, 11, 11, 128)      

In [44]:
history = model.fit(
    train_ds,
    steps_per_epoch=int(np.ceil(TRAIN_IMG_COUNT // float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_ds,
    validation_steps=int(np.ceil(VAL_IMG_COUNT // float(BATCH_SIZE))),
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler]
)

Epoch 1/35
182/182 [==============================] - 851s 4s/step - loss: 0.6623 - accuracy: 0.6659 - precision: 0.7447 - recall: 0.7853 - val_loss: 27.0476 - val_accuracy: 0.6822 - val_precision: 0.6822 - val_recall: 1.0000
Epoch 2/35
182/182 [==============================] - 18s 100ms/step - loss: 0.4666 - accuracy: 0.8126 - precision: 0.8195 - recall: 0.9313 - val_loss: 2.8398 - val_accuracy: 0.6830 - val_precision: 0.6830 - val_recall: 1.0000
Epoch 3/35
182/182 [==============================] - 18s 98ms/step - loss: 0.4562 - accuracy: 0.8107 - precision: 0.8198 - recall: 0.9235 - val_loss: 1.6475 - val_accuracy: 0.6830 - val_precision: 0.6830 - val_recall: 1.0000
Epoch 4/35
182/182 [==============================] - 18s 101ms/step - loss: 0.4574 - accuracy: 0.8133 - precision: 0.8277 - recall: 0.9188 - val_loss: 3.1056 - val_accuracy: 0.6822 - val_precision: 0.6822 - val_recall: 1.0000
Epoch 5/35
182/182 [==============================] - 18s 100ms/step - loss: 0.4630 - accuracy

In [46]:
model.evaluate(test_ds)

32/32 [==============================] - 111s 3s/step - loss: 0.5040 - accuracy: 0.7377 - precision: 0.8964 - recall: 0.6998


[0.5039916634559631, 0.737687349319458, 0.8964143395423889, 0.699844479560852]

In [47]:
model.save('./Breast_Cancer_tf',save_format='tf')
!zip -r Breast_Cancer.zip {'./Breast_Cancer_tf'}

  adding: Breast_Cancer_tf/ (stored 0%)
  adding: Breast_Cancer_tf/variables/ (stored 0%)
  adding: Breast_Cancer_tf/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: Breast_Cancer_tf/variables/variables.index (deflated 75%)
  adding: Breast_Cancer_tf/assets/ (stored 0%)
  adding: Breast_Cancer_tf/saved_model.pb (deflated 90%)
